In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
pd.set_option('display.max_columns', 500)

/Users/Enzo.Calogero/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float32',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float32',
        'AVProductsEnabled':                                    'float32',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float32',
        'GeoNameIdentifier':                                    'float32',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float32',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float32',
        'IeVerIdentifier':                                      'float32',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float32',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float32',
        'Census_ProcessorManufacturerIdentifier':               'float32',
        'Census_ProcessorModelIdentifier':                      'float32',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float32',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float32',
        'Census_IsFlightsDisabled':                             'float32',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float32',
        'Census_FirmwareManufacturerIdentifier':                'float32',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float32',
        'Census_IsVirtualDevice':                               'float32',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float32',
        'Wdft_IsGamer':                                         'float32',
        'Wdft_RegionIdentifier':                                'float32',
        'HasDetections':                                        'int8'
        }

In [ ]:
nrows = 2000000
#_______________________________________________________________________________
#numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#numerical_columns = [c for c,v in dtypes.items() if v in numerics]
#categorical_columns = [c for c,v in dtypes.items() if v not in numerics]
categorical_columns = [
    'EngineVersion',
    'AppVersion',
    'AvSigVersion',
    'Platform',
    'Processor',
    'OsVer',
    'OsPlatformSubRelease',
    'OsBuildLab',
    'SkuEdition',
    'PuaMode',
    'SmartScreen',
    'Census_MDC2FormFactor',
    'Census_DeviceFamily',
    'Census_ProcessorClass',
    'Census_PrimaryDiskTypeName',
 ]

In [3]:
#retained_columns = numerical_columns + categorical_columns
train = pd.read_csv('train.csv',
                    
                    #usecols = retained_columns,
                    dtype = dtypes)
#nrows = nrows,
#retained_columns += ['MachineIdentifier']
#retained_columns.remove('HasDetections')

#train=train.drop(['MachineIdentifier'])



In [3]:
test = pd.read_csv('test.csv',
                   #usecols = retained_columns,
                   dtype = dtypes)

In [4]:
train.head(1)

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,-85,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,2341.0,NaN,476940.0,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0


In [5]:
datedict = np.load('AvSigVersionTimestamps.npy')
datedict = datedict[()]
# ADD TIMESTAMPS
train['Date'] = train['AvSigVersion'].map(datedict)

In [6]:
test.info()

NameError: name 'test' is not defined

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8921483 entries, 0 to 8921482
Data columns (total 84 columns):
MachineIdentifier                                    category
ProductName                                          category
EngineVersion                                        category
AppVersion                                           category
AvSigVersion                                         category
IsBeta                                               int8
RtpStateBitfield                                     float32
IsSxsPassiveMode                                     int8
DefaultBrowsersIdentifier                            float32
AVProductStatesIdentifier                            float32
AVProductsInstalled                                  float32
AVProductsEnabled                                    float32
HasTpm                                               int8
CountryIdentifier                                    int16
CityIdentifier                         

In [8]:
def engine(row):
    listmia=str(row['EngineVersion']).split('.')
    if len(listmia) > 1:
        return float(str( str(listmia[2])+ '.' + str(listmia[3])))
                     #listmia[0])+  str(listmia[1])+ symplified version
    return 0


In [9]:
train['enzoEngine']=train.apply(lambda row: engine(row), axis=1)
#train['EngineVersion'] = train['EngineVersion'].astype('category')

In [6]:
test['enzoEngine']=test.apply(lambda row: engine(row), axis=1)
#test['EngineVersion'] = test['EngineVersion'].astype('category')

In [10]:
train.head().T

,0,1,2,3,4
MachineIdentifier,0000028988387b115f69f31a3bf04f09,000007535c3f730efa9ea0b7ef1bd645,000007905a28d863f6d0d597892cd692,00000b11598a75ea8ba1beea8459149f,000014a5f00daa18e76b81417eeb99fc
ProductName,win8defender,win8defender,win8defender,win8defender,win8defender
EngineVersion,1.1.15100.1,1.1.14600.4,1.1.15100.1,1.1.15100.1,1.1.15100.1
AppVersion,4.18.1807.18075,4.13.17134.1,4.18.1807.18075,4.18.1807.18075,4.18.1807.18075
AvSigVersion,1.273.1735.0,1.263.48.0,1.273.1341.0,1.273.1527.0,1.273.1379.0
IsBeta,0,0,0,0,0
RtpStateBitfield,7,7,7,7,7
IsSxsPassiveMode,0,0,0,0,0
DefaultBrowsersIdentifier,NaN,NaN,NaN,NaN,NaN
AVProductStatesIdentifier,53447,53447,53447,53447,53447


In [11]:
def AppV(row):
    listmia=str(row['AppVersion']).split('.')
    if len(listmia) > 1:
        return(float("{:0>2}{:0>5}.{}".format(str(listmia[1]),str(listmia[2]),str(listmia[3]))))
    #symplified take away the initial 4...
    return 0

In [12]:
train['Enzoapp']=train.apply(lambda row: AppV(row), axis=1)
#train['AppVersion'] = train['AppVersion'].astype('category')

In [8]:
test['Enzoapp']=test.apply(lambda row: AppV(row), axis=1)
#test['AppVersion'] = test['AppVersion'].astype('category')

In [13]:
train.head().T

,0,1,2,3,4
MachineIdentifier,0000028988387b115f69f31a3bf04f09,000007535c3f730efa9ea0b7ef1bd645,000007905a28d863f6d0d597892cd692,00000b11598a75ea8ba1beea8459149f,000014a5f00daa18e76b81417eeb99fc
ProductName,win8defender,win8defender,win8defender,win8defender,win8defender
EngineVersion,1.1.15100.1,1.1.14600.4,1.1.15100.1,1.1.15100.1,1.1.15100.1
AppVersion,4.18.1807.18075,4.13.17134.1,4.18.1807.18075,4.18.1807.18075,4.18.1807.18075
AvSigVersion,1.273.1735.0,1.263.48.0,1.273.1341.0,1.273.1527.0,1.273.1379.0
IsBeta,0,0,0,0,0
RtpStateBitfield,7,7,7,7,7
IsSxsPassiveMode,0,0,0,0,0
DefaultBrowsersIdentifier,NaN,NaN,NaN,NaN,NaN
AVProductStatesIdentifier,53447,53447,53447,53447,53447


In [12]:
train=train.drop(index=5244810)

In [14]:
def SignV(row):
    listmia=str(row['AvSigVersion']).split('.')
    if len(listmia) > 3:
        return(float("{:0>2}{:0>3}{:0>4}.{}".format(str(listmia[0]),str(listmia[1]),str(listmia[2]),str(listmia[3]))))
    return 0

In [15]:
train['Enzosign']=train.apply(lambda row: SignV(row), axis=1)

ValueError: ("could not convert string to float: '012&#x17;31144.0'", 'occurred at index 5244810')

In [10]:
test['Enzosign']=test.apply(lambda row: SignV(row), axis=1)

In [ ]:
train.head().T

In [ ]:
def func1(row):
    listmia=str(row['OsBuildLab']).split('.')
    if len(listmia) > 1:
        return int(listmia[0])
    return 0

def func2(row):
    listmia=str(row['OsBuildLab']).split('.')
    if len(listmia) > 3:
        return int(listmia[1].split('*')[0].split('*')[0])
    return 0

def func3(row):
    listmia=str(row['OsBuildLab']).split('.')
    if len(listmia) > 1:
        return listmia[2]
    return '___'

def func4(row):
    listmia=str(row['OsBuildLab']).split('.')
    if len(listmia) > 1:
        return listmia[3]
    return '___'

In [ ]:
train['first']=train.apply(lambda row: func1(row), axis=1)
train['second']=train.apply(lambda row: func2(row), axis=1)
train['third']=train.apply(lambda row: func3(row), axis=1)
train['fourth']=train.apply(lambda row: func4(row), axis=1)

In [12]:
test['first']=test.apply(lambda row: func1(row), axis=1)
test['second']=test.apply(lambda row: func2(row), axis=1)
test['third']=test.apply(lambda row: func3(row), axis=1)
test['fourth']=test.apply(lambda row: func4(row), axis=1)

In [ ]:
#skip for now
train['third']=train['third'].astype('category')
train['fourth']=train['fourth'].astype('category')
#train['OsBuildLab']=train['OsBuildLab'].astype('category')

train.head().T

In [ ]:
skip for now
test['third']=test['third'].astype('category')
test['fourth']=test['fourth'].astype('category')
test['OsBuildLab']=test['OsBuildLab'].astype('category')
test.head().T

In [ ]:
def func5(row):
    intero=row['first']
    decimale=row['second']
    return float("{}.{}".format(int(intero),int(decimale)))

In [ ]:
train['fifth']=train.apply(lambda row: func5(row), axis=1)
#train['fifth']=train['fifth'].astype('category')

In [14]:
test['fifth']=test.apply(lambda row: func5(row), axis=1)
#test['fifth']=test['fifth'].astype('category')

In [ ]:
to be added
train['third']=train['third'].astype('category')
train['fourth']=train['fourth'].astype('category')
test['third']=test['third'].astype('category')
test['fourth']=test['fourth'].astype('category')

In [ ]:
test.head(1)

In [15]:
import os
import feather
os.makedirs('tmp', exist_ok=True)
feather.write_dataframe(test, 'tmp/microsoft_test_prep_v2_temp')

In [ ]:
import os
import feather
os.makedirs('tmp', exist_ok=True)
feather.write_dataframe(train, 'tmp/microsoft_train_prep_v2b_temp')

In [2]:
import os
import feather
test =feather.read_dataframe('tmp/microsoft_test_prep_v2_temp')

In [3]:
import os
import feather
train =feather.read_dataframe('tmp/microsoft_train_prep_v2b_temp')

ArrowIOError: Failed to open local file: tmp/microsoft_train_prep_v2b_temp , error: No such file or directory

In [ ]:
train.head(1)

In [ ]:
train.info()

In [17]:
train=train.drop(['MachineIdentifier'],axis=1)
categorical_columns = list()
for col in train.columns:
    if isinstance(train[col].dtype, pd.api.types.CategoricalDtype):
        categorical_columns.append(col)
print(categorical_columns)
print(len(categorical_columns))

['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'Platform', 'Processor', 'OsVer', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'PuaMode', 'SmartScreen', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSVersion', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_FlightRing']
29


In [16]:
import os
import feather
train =feather.read_dataframe('tmp/microsoft_train_prep_v2_temp')

In [ ]:
training...
>>> labels, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'])
>>> labels
array([0, 0, 1, 2, 0])
>>> uniques
array(['b', 'a', 'c'], dtype=object)

£££££££££
#modify the index

>>> indexer = index.get_indexer(new_index)
>>> new_values = cur_values.take(indexer)

In [18]:
indexer = {}
for col in categorical_columns:
    _, indexer[col] = pd.factorize(train[col])
for col in categorical_columns:
    train[col] = indexer[col].get_indexer(train[col])
    test[col] = indexer[col].get_indexer(test[col])    

In [19]:
import os
import feather
os.makedirs('tmp', exist_ok=True)
feather.write_dataframe(train, 'tmp/microsoft_train_prep_v2_fullb')

#train.to_feather('tmp/bulldozers-raw')

In [20]:
import os
import feather
os.makedirs('tmp', exist_ok=True)

feather.write_dataframe(test, 'tmp/microsoft_test_prep_v2_fullb')

In [ ]:
test.head().T


In [ ]:
train.head().T

In [ ]:
train.info()